## 병행성 Concurrency
** 왜 병행성을? **
- 성능(perfomance): 느린 요소(Component)를 기다리지 않고, 빠른 요소를 바쁘게 유지한다.
- 견고함(Robustness): 하드웨어 및 소프트웨어의 장애를 피하기 위해 작업을 복제하여 여러가지 안정적인 방식으로 운영한다.
- 간소화(Simplicity): 복잡한 작업을 좀 더 이해하기 쉽고, 해결하기 쉬운 여러 작은 작업으로 분해한다.
- 커뮤니케이션(Communication): 데이터(바이트)를 보내고 싶은 곳에 원격으로 전송하고, 데이터를 다시 수신 받는다.

** 컴퓨터가 일을 수행하면서 뭔가 기다린다면, 두 가지 이유중 하나이다.**
- I/O 바운드: CPU는 정말 빠르기 때문에 보통 이 경우에 해당함
- CPU 바운드: 정말엄청데~단한계산할때

** 다음의 용어는 병행성과 관련이 있다.**
- 동기(Synchronous): 한 작업은 다른 작업을 따른다.
- 비동기(Asynchronous): 작업들이 각각 독립적이다.

### Queue
** FIFO **<br>
** 분산 작업 관리를 위한 큐의 경우 작업 큐_(work queue, job queue, task queue)_라고 알려져 있음**

### Process
**Queue를 구현하는 여러가지 방법 중 한 가지 방법**

In [4]:
import multiprocessing as mp

def washer(dishes, output):
    for dish in dishes:
        print('Washing', dish, 'dish')
        output.put(dish)

def dryer(input):
    while True:
        dish = input.get()
        print('Drying', dish, 'dish')
        input.task_done()
        
dish_queue = mp.JoinableQueue()
dryer_proc = mp.Process(target=dryer, args=(dish_queue,))
dryer_proc.daemon = True
dryer_proc.start()

dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)
dish_queue.join()

Drying salad dish
Drying bread dish
Drying entree dish
Drying dessert dish
Washing salad dish
Washing bread dish
Washing entree dish
Washing dessert dish


**이터레이터와 매우 유사**

### Thread
** Thread는 한 프로세스 내에서 실행 됨 **<br>
** 프로세스의 모든 자원에 접근할 수 있음**

In [5]:
import threading

def do_this(what):
    whoami(what)
    
def whoami(what):
    print("Thread %s says: %s" % (threading.current_thread(), what))
    
if __name__ == "__main__":
    whoami("I'm the main program")
    for n in range(4):
        p = threading.Thread(target=do_this, args=("I'm function %s" % n,))
        p.start()

Thread <_MainThread(MainThread, started 140646213403776)> says: I'm the main program
Thread <Thread(Thread-4, started 140645901506304)> says: I'm function 0
Thread <Thread(Thread-5, started 140645482100480)> says: I'm function 1Thread <Thread(Thread-6, started 140645901506304)> says: I'm function 2

Thread <Thread(Thread-7, started 140645482100480)> says: I'm function 3


** 프로세스 기반의 윗 예제를 스레드로 구현하자. **

In [6]:
import threading, queue
import time

def washer(dishes, dish_queue):
    for dish in dishes:
        print("Washing", dish)
        time.sleep(5)
        dish_queue.put(dish)
        
def dryer(dish_queue):
    while True:
        dish = dish_queue.get()
        print("Drying", dish)
        time.sleep(10)
        dish_queue.task_done()

dish_queue = queue.Queue()
for n in range(2):
    dryer_thread = threading.Thread(target=dryer, args=(dish_queue,))
    dryer_thread.start()
    
dishes = ['salad', 'bread', 'entree', 'desert']
washer(dishes, dish_queue)
dish_queue.join()

Washing salad
WashingDrying salad
 bread
Drying Washingbread 
entree
Drying Washingentree 
desert
Drying desert


** _스레드는 전역 데이터가 관여하지 않을 때 유용하고 안전하다._ **<br>
** I/O 바운드 문제 - 스레드 사용, CPU 바운드 문제 - 프로세스, 네트워킹, 이벤트 사용 **

### 그린 스레드와 gevent
**Gevent**

In [1]:
import gevent
from gevent import socket
hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com', 'www.antique-taxidermy.com']
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)

66.6.44.4
172.217.25.243
None


**_gevent.spawn()_은 각각의 _gevent.socket.gethostbyname(host)_를 실행하기 위해 greenlet(_green thread/micro thread_)를 생성**<br>
greenlet과 일반적인 스레드의 차이점은 Block을 하지 않는다는 것이다. 한 스레드에서 무슨 일로 블록 되었다면, gevent는 제어를 다른 하나의 greenlet으로 바꾼다. <br>
**_gevent.joinall()_ 메소드는 생성된 모든 작업이 끝날 때까지 기다린다. 그리고 호스트 네임에 대한 IP 주소를 한 번에 얻게 된다. **<br>
**gevent 버전의 socket 대신, 기억하기 쉬운 이름의 몽키-패치(_monkey-patch_) 함수 사용 가능**<br>
gevent 버전의 모듈을 호출하지 않고, greenlet을 사용하기 위해 socket과 같은 표준 모듈을 수정한다.

In [3]:
import gevent
from gevent import monkey; monkey.patch_all()
import socket
hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com', 'www.antique-taxidermy.com']
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)


The history saving thread hit an unexpected error (LoopExit('This operation would block forever', <Hub at 0x7f51c7563df0 epoll pending=0 ref=0 fileno=57>)).History will not be written to the database.
66.6.44.4
172.217.26.19
None


** gevent의 잠재적 위험**<br>
- 모든 이벤트 기반 시스템에서, 실행하는 각 코드 단위는 상대적으로 빠르게 처리되어야 함
- 논블로킹_nonblocking_임에도 불구하고 많은 일을 처리해야 하는 코드는 여전히 느

### asyncio
** 많은 패키지는 자신의 이벤트 루프를 가지며, 각 이벤트 루프는 자신이 유일하길 원함**
**'비동기 입출력 지원 재정리: asyncio 모듈(_Asynchronous IO Support Rebooted_) 코드 이름 tulip**

https://docs.python.org/3/library/asyncio.html

분노에 메이지쨘은 이 세줄을 참을수가 업엇어요 그래서 전문가 파이썬 책을 차자옴
**Fluent Python** 폴더 참